In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import pickle
import tensorflow as tf
from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers import Dense



2023-07-06 18:24:13.928160: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-06 18:24:14.594760: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-06 18:24:14.793933: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-06 18:24:14.793956: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

# IMPORT DATA

In [4]:
folder_path_train = "../data/Match_Diff"

In [5]:
data_dict = {}

# Iterate over files in the folder
for filename in os.listdir(folder_path_train):
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path_train, filename)
        
        # Extract the key from the filename
        key = filename.replace("Match_Diff_", "").replace(".csv", "")
        
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        
        # Delete the first column from the DataFrame
        df = df.drop(df.columns[0], axis=1)
        
        # Add the DataFrame to the dictionary with the key
        data_dict[key] = df

# STARTS MODEL

In [6]:
from get_json import reduce_columns_train

data_dict_red={}

for key, df in data_dict.items():
    
    data_dict_red[key] = reduce_columns_train(data_dict[key])
    


In [7]:
from clean_preprocess import preprocess_train

data_preprop = {}

for key, df in data_dict_red.items():
    
    X = data_dict_red[key].drop(columns="target")
    
    y = data_dict_red[key]["target"]

    X_preprop,transformer = preprocess_train(X)
    
    data_preprop[key] = [X_preprop,y,transformer]

In [8]:
columns_of_interest = data_preprop["IRON"][0].columns.tolist()
columns_of_interest

['killType_KILL_ACE',
 'killType_KILL_FIRST_BLOOD',
 'killType_KILL_MULTI',
 'minionsKilled',
 'monsterType_AIR_DRAGON',
 'monsterType_CHEMTECH_DRAGON',
 'monsterType_EARTH_DRAGON',
 'monsterType_FIRE_DRAGON',
 'monsterType_HEXTECH_DRAGON',
 'monsterType_RIFTHERALD',
 'monsterType_WATER_DRAGON',
 'totalGold',
 'towerType_INNER_TURRET',
 'towerType_OUTER_TURRET']

# LOG MODEL

In [9]:
#Look for the best params and export the model with these

fitted_models = {}

for key, value in data_preprop.items():
    print(key)

    X_preprop = data_preprop[key][0]
    y = data_preprop[key][1]

    X_train, X_test, y_train, y_test = train_test_split(X_preprop, y, test_size=0.2, random_state=42)

    param_grid = {
        'C': [0.1, 1, 10],
        'penalty': ['l1', 'l2'],
        'solver': ['liblinear', 'saga']
    }

    model = LogisticRegression(max_iter=5000)

    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring='accuracy',
        cv=5
    )

    grid_search.fit(X_train, y_train)

    print("Best Hyperparameters: ", grid_search.best_params_)
    print("Best Score: ", grid_search.best_score_)

    best_model = LogisticRegression(
        C=grid_search.best_params_['C'],
        penalty=grid_search.best_params_['penalty'],
        solver=grid_search.best_params_['solver'],
        max_iter=5000
    )
    best_model.fit(X_train, y_train)

    # Save the fitted model in the dictionary using the key as the key
    fitted_models[key] = best_model

    # Export the model as a pickle file. Uncoment if you want
    """
    filename = key + '_model.pkl'
    with open(filename, 'wb') as file:
        pickle.dump(best_model, file)
    print("Model exported as", filename)
    """

IRON
Best Hyperparameters:  {'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'}
Best Score:  0.7357476220185917
PLATINUM
Best Hyperparameters:  {'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'}
Best Score:  0.7179814586473033
SILVER
Best Hyperparameters:  {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
Best Score:  0.7126429818155621
GOLD
Best Hyperparameters:  {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
Best Score:  0.7137599820730117
BRONZE
Best Hyperparameters:  {'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}
Best Score:  0.7025020061905308
CHALLENGER
Best Hyperparameters:  {'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'}
Best Score:  0.7233133473241978
MASTER
Best Hyperparameters:  {'C': 1, 'penalty': 'l1', 'solver': 'saga'}
Best Score:  0.7052763483570029
GRANDMASTER
Best Hyperparameters:  {'C': 0.1, 'penalty': 'l1', 'solver': 'saga'}
Best Score:  0.7123748584985016
DIAMOND
Best Hyperparameters:  {'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}
Best Score:  0.70438190

# PREDICTION

In [10]:
pred_folder = "../data/predict"

In [11]:
from get_json import process_folder
from get_diff import calculate_event_differences
from clean_preprocess import preprocess_pred
from get_json import check_and_create_columns
from get_json import check_and_create_columns


In [14]:
minute = 10
look_events=["CHAMPION_SPECIAL_KILL","CHAMPION_KILL","ELITE_MONSTER_KILL","BUILDING_KILL"] 
folder_path = pred_folder

league="IRON"

def prediction(pred_folder,minute,look_events,columns_of_interest,league):

    df = process_folder(pred_folder,minute,look_events)
    df_dif = calculate_event_differences(df)
    df_dif.drop(columns="matchId",inplace=True)
    df_cc = check_and_create_columns(df_dif, columns_of_interest)
    
    X_pred_prep = preprocess_pred(df_cc,data_preprop[str(league)][2])

    model = fitted_models[str(league)]
    
    results = model.predict_proba(X_pred_prep)

    return results


/home/santiagop/code/nicolas-muzzio/Spirit_of_the_LoL/notebooks/get_diff.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_events_diff = all_events_diff.append(resta_filas, ignore_index=True)


array([[0.39876417, 0.60123583]])

In [18]:
prediction(pred_folder,minute,look_events,columns_of_interest,league)[0][1]

/home/santiagop/code/nicolas-muzzio/Spirit_of_the_LoL/notebooks/get_diff.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_events_diff = all_events_diff.append(resta_filas, ignore_index=True)


0.60123582850202